In [16]:
import geopandas as gpd
from typing import List, Tuple

def find_intersecting_polygons(polygons_orig: gpd.GeoDataFrame, points: gpd.GeoDataFrame) -> Tuple[
    gpd.GeoDataFrame, List[str]]:
    mixed_segments = []
    polygons = polygons_orig.copy()
    intersections = gpd.sjoin(polygons, points, how='inner', predicate='intersects')

    for polygon_id, group in intersections.groupby(intersections.index):
        classes = group['class'].unique()

        if len(classes) == 1:
            polygons.loc[polygon_id, 'feature_class'] = classes[0]
        else:
            segment_id = group['segment_id'].values[0]
            mixed_segments.append(segment_id)

    polygons = polygons[polygons['feature_class'].notna()]

    return polygons, mixed_segments


In [4]:
points = gpd.read_file("/mnt/c/tmp/training_points.gpkg")

In [10]:
polygons_orig = gpd.read_file("/mnt/c/tmp/output/output_62.gpkg")

In [11]:
polygons_orig.head()

,segment_id,feature_class,b0_mean,b0_variance,b0_contrast,b0_dissimilarity,b0_energy,b0_correlation,b1_mean,b1_variance,...,b6_dissimilarity,b6_energy,b6_correlation,b7_mean,b7_variance,b7_contrast,b7_dissimilarity,b7_energy,b7_correlation,geometry
0,1,None,32.884058,3.928586,0.014732,0.000470,0.999978,0.544983,35.057971,3.445915,...,0.000704,0.999978,0.542783,43.869565,6.287335,0.027180,0.000636,0.999978,0.531012,"POLYGON ((194884.608 2190240.388, 194884.608 2..."
1,2,None,34.759494,15.018106,0.031577,0.000929,0.999967,0.348087,40.493671,13.591732,...,0.001900,0.999967,0.345300,66.164557,61.605832,0.115187,0.001790,0.999967,0.344746,"POLYGON ((194890.831 2190240.388, 194890.831 2..."
2,3,None,46.016393,25.917764,0.048307,0.001059,0.999974,0.262905,55.196721,25.633432,...,0.002384,0.999974,0.256487,94.147541,37.240527,0.201387,0.002137,0.999974,0.260033,"POLYGON ((194894.661 2190240.388, 194894.661 2..."
3,4,None,39.031915,14.137279,0.039845,0.001045,0.999964,0.450024,46.946809,17.412064,...,0.002296,0.999964,0.452636,79.212766,47.444092,0.163330,0.002113,0.999964,0.451687,"POLYGON ((194898.969 2190240.388, 194898.969 2..."
4,5,None,43.504673,5.053716,0.048211,0.001138,0.999960,0.526318,53.280374,6.126998,...,0.002751,0.999960,0.525067,97.897196,15.438030,0.243861,0.002550,0.999960,0.526332,"POLYGON ((194902.320 2190240.388, 194902.320 2..."


In [17]:
polygons, mixed_segments = find_intersecting_polygons(polygons_orig, points)

In [18]:
len(mixed_segments)

0

In [19]:
print(polygons.shape)
polygons.head()

(109, 51)


,segment_id,feature_class,b0_mean,b0_variance,b0_contrast,b0_dissimilarity,b0_energy,b0_correlation,b1_mean,b1_variance,...,b6_dissimilarity,b6_energy,b6_correlation,b7_mean,b7_variance,b7_contrast,b7_dissimilarity,b7_energy,b7_correlation,geometry
8062,8063,9,21.500000,58.284483,0.010942,0.000464,0.999976,0.276928,21.396552,74.377229,...,0.001805,0.999976,0.316557,78.586207,181.725327,0.127585,0.001620,0.999976,0.309708,"POLYGON ((195776.899 2190086.248, 195776.899 2..."
8480,8481,9,15.708861,78.105111,0.008513,0.000438,0.999969,0.338272,14.772152,131.365807,...,0.001493,0.999969,0.412602,55.835443,143.377984,0.075897,0.001367,0.999969,0.412237,"POLYGON ((195803.227 2190078.110, 195803.227 2..."
10507,10508,9,20.546512,67.666441,0.015077,0.000656,0.999966,0.285945,20.023256,97.138994,...,0.002344,0.999966,0.367274,80.337209,281.990941,0.181888,0.002230,0.999966,0.373599,"POLYGON ((195784.558 2190038.857, 195784.558 2..."
14268,14269,9,26.906250,83.397461,0.026395,0.000896,0.999963,0.320558,29.187500,125.256510,...,0.002691,0.999963,0.366228,81.614583,236.299371,0.206929,0.002482,0.999963,0.376511,"POLYGON ((195744.826 2189965.616, 195744.826 2..."
23237,23238,9,29.328571,101.792041,0.022693,0.000715,0.999972,0.327655,34.571429,143.473469,...,0.002330,0.999972,0.381357,95.500000,147.878571,0.197062,0.002090,0.999972,0.394013,"POLYGON ((195765.410 2189791.371, 195765.410 2..."


In [ ]:
polygons.to_file("output_with_classes.gpkg", driver="GPKG")